<a href="https://colab.research.google.com/github/ameyawcharles/Animation-Nation/blob/master/zeroscope_v2_576w_text_to_video_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%cd /content
!pip install diffusers==0.19.0 transformers accelerate imageio[ffmpeg] -U einops omegaconf decord safetensors
!pip install -q https://download.pytorch.org/whl/cu121/xformers-0.0.22.post7-cp310-cp310-manylinux2014_x86_64.whl
!git clone -b dev https://github.com/camenduru/Text-To-Video-Finetuning

!apt -y install -qq aria2
model = "cerspense/zeroscope_v2_576w"
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/{model}/raw/main/model_index.json -d /content/model -o model_index.json
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/{model}/raw/main/vae/config.json -d /content/model/vae -o config.json
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/{model}/resolve/main/vae/diffusion_pytorch_model.bin -d /content/model/vae -o diffusion_pytorch_model.bin
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/{model}/raw/main/unet/config.json -d /content/model/unet -o config.json
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/{model}/resolve/main/unet/diffusion_pytorch_model.bin -d /content/model/unet -o diffusion_pytorch_model.bin
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/{model}/raw/main/text_encoder/config.json -d /content/model/text_encoder -o config.json
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/{model}/resolve/main/text_encoder/pytorch_model.bin -d /content/model/text_encoder -o pytorch_model.bin
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/{model}/raw/main/scheduler/scheduler_config.json -d /content/model/scheduler -o scheduler_config.json
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/{model}/raw/main/tokenizer/merges.txt -d /content/model/tokenizer -o merges.txt
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/{model}/raw/main/tokenizer/special_tokens_map.json -d /content/model/tokenizer -o special_tokens_map.json
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/{model}/raw/main/tokenizer/tokenizer_config.json -d /content/model/tokenizer -o tokenizer_config.json
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/{model}/raw/main/tokenizer/vocab.json -d /content/model/tokenizer -o vocab.json

/content
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 960.5/960.5 kB 41.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 129.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.6/13.6 MB 125.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 131.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 95.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 63.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 M

In [ ]:
%cd /content/Text-To-Video-Finetuning
# while True:
prompt = "waterfall" #@param {type:"string"}
negative = "text, watermark, copyright, blurry, low resolution, blur, low quality" #@param {type:"string"}
prompt = f"\"{prompt}\""
negative = f"\"{negative}\""
num_steps = 25 #@param {type:"raw"}
guidance_scale = 23 #@param {type:"raw"}
fps = 24 #@param {type:"raw"}
num_frames = 10 #@param {type:"raw"}
!python inference.py -m "/content/model" -p {prompt} -n {negative} -W 576 -H 320 -o /content/outputs -d cuda -x -s {num_steps} -g {guidance_scale} -f {fps} -T {num_frames}